In [1]:
import glob
import json
import os
from collections import defaultdict
from pathlib import Path

import awkward as ak
import dask
import dask_awkward as dak
import hist.dask
import coffea
import numpy as np
import uproot
from dask.distributed import Client

from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema
from coffea import dataset_tools

import time
import warnings

# from servicex import ServiceXDataset
from servicex import ServiceXSpec, General, Sample
from servicex.uproot_raw.uproot_raw import UprootRawQuery
from servicex.func_adl.func_adl_dataset import FuncADLQuery
from func_adl_servicex_xaodr22 import FuncADLQueryPHYSLITE
from func_adl_servicex_xaodr22 import (
    cpp_float,
    cpp_int,
    cpp_vfloat,
    cpp_vint,
)
from servicex.servicex_client import deliver
import json

warnings.filterwarnings("ignore")

from input_files import utils

from dask.distributed import LocalCluster, Client, progress, performance_report

# local: single thread, single worker
cluster = LocalCluster(n_workers=1, processes=False, threads_per_worker=5)
client = Client(cluster)

# for UChicago
# client = Client("tcp://dask-alheld-c58c6d0f-b.af-jupyter:8786")  # update this to point to your own client!

print(f"awkward: {ak.__version__}")
print(f"dask-awkward: {dak.__version__}")
print(f"uproot: {uproot.__version__}")
print(f"hist: {hist.__version__}")
print(f"coffea: {coffea.__version__}")

awkward: 2.6.2
dask-awkward: 2024.3.0
uproot: 5.3.2
hist: 2.7.2
coffea: 2024.3.0


In [2]:
def strip_xcache(fpath):
    import urllib
    while True:
        parsed = urllib.parse.urlparse(fpath)
        if not parsed.scheme:
            return parsed.geturl()
        fpath = parsed.path
    return fpath

In [3]:
# defined for direct access ??
BRANCH_LIST = [
    "AnalysisJetsAuxDyn.pt", "AnalysisJetsAuxDyn.eta", "AnalysisJetsAuxDyn.phi", "AnalysisJetsAuxDyn.m",
    "AnalysisElectronsAuxDyn.pt", "AnalysisElectronsAuxDyn.eta", "AnalysisElectronsAuxDyn.phi",
    "AnalysisElectronsAuxDyn.m", "AnalysisMuonsAuxDyn.pt", "AnalysisMuonsAuxDyn.eta",
    "AnalysisMuonsAuxDyn.phi", 
    # "AnalysisJetsAuxDyn.EnergyPerSampling", # BAD! uproot can't write var * var * float32 to ROOT ??
    # "AnalysisJetsAuxDyn.SumPtTrkPt500",  # BAD! uproot can't write var * var * float32 to ROOT ??
    # "AnalysisJetsAuxDyn.TrackWidthPt1000",  # BAD! uproot can't write var * var * float32 to ROOT ??
    "PrimaryVerticesAuxDyn.z", "PrimaryVerticesAuxDyn.x",
    "PrimaryVerticesAuxDyn.y", 
    # "AnalysisJetsAuxDyn.NumTrkPt500", "AnalysisJetsAuxDyn.NumTrkPt1000", # BAD! uproot can't write var * var * int32 to ROOT ??
    # "AnalysisJetsAuxDyn.SumPtChargedPFOPt500", # BAD! uproot can't write var * var * float32 to ROOT ??
    "AnalysisJetsAuxDyn.Timing",
    "AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta", "AnalysisJetsAuxDyn.ActiveArea4vec_eta",
    "AnalysisJetsAuxDyn.DetectorEta", "AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi",
    "AnalysisJetsAuxDyn.ActiveArea4vec_phi", "AnalysisJetsAuxDyn.JetConstitScaleMomentum_m",
    "AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt", "AnalysisJetsAuxDyn.EMFrac",
    "AnalysisJetsAuxDyn.Width", "AnalysisJetsAuxDyn.ActiveArea4vec_m", "AnalysisJetsAuxDyn.ActiveArea4vec_pt",
    "AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksWidth", "AnalysisJetsAuxDyn.PSFrac",
    "AnalysisJetsAuxDyn.JVFCorr", "AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksC1",
    "AnalysisJetsAuxDyn.DFCommonJets_fJvt", "AnalysisJetsAuxDyn.DFCommonJets_QGTagger_NTracks",
    "AnalysisJetsAuxDyn.GhostMuonSegmentCount", 
    # "AnalysisMuonsAuxDyn.muonSegmentLinks*", # BAD! uproot can't write var * var * struct[{m_persKey: uint32, m_persIndex: uint32}]
    # "AnalysisMuonsAuxDyn.msOnlyExtrapolatedMuonSpectrometerTrackParticleLink*", # Links don't crash the job but don't show up in output either
    # "AnalysisMuonsAuxDyn.extrapolatedMuonSpectrometerTrackParticleLink*",
    # "AnalysisMuonsAuxDyn.inDetTrackParticleLink*", "AnalysisMuonsAuxDyn.muonSpectrometerTrackParticleLink*",
    "AnalysisMuonsAuxDyn.momentumBalanceSignificance", "AnalysisMuonsAuxDyn.topoetcone20_CloseByCorr",
    "AnalysisMuonsAuxDyn.scatteringCurvatureSignificance", "AnalysisMuonsAuxDyn.scatteringNeighbourSignificance",
    "AnalysisMuonsAuxDyn.neflowisol20_CloseByCorr", "AnalysisMuonsAuxDyn.topoetcone20",
    "AnalysisMuonsAuxDyn.topoetcone30", "AnalysisMuonsAuxDyn.topoetcone40", "AnalysisMuonsAuxDyn.neflowisol20",
    "AnalysisMuonsAuxDyn.segmentDeltaEta", "AnalysisMuonsAuxDyn.DFCommonJetDr",
    # "AnalysisMuonsAuxDyn.combinedTrackParticleLink*", 
    "AnalysisMuonsAuxDyn.InnerDetectorPt",
    "AnalysisMuonsAuxDyn.MuonSpectrometerPt", #"AnalysisMuonsAuxDyn.clusterLink*",
    "AnalysisMuonsAuxDyn.spectrometerFieldIntegral", #"AnalysisElectronsAuxDyn.ambiguityLink*",
    "AnalysisMuonsAuxDyn.EnergyLoss", "AnalysisJetsAuxDyn.NNJvtPass", "AnalysisElectronsAuxDyn.topoetcone20_CloseByCorr",
    "AnalysisElectronsAuxDyn.topoetcone20ptCorrection", "AnalysisElectronsAuxDyn.topoetcone20",
    "AnalysisMuonsAuxDyn.ptvarcone30_Nonprompt_All_MaxWeightTTVA_pt500_CloseByCorr",
    "AnalysisElectronsAuxDyn.DFCommonElectronsECIDSResult", "AnalysisElectronsAuxDyn.neflowisol20",
    "AnalysisMuonsAuxDyn.ptvarcone30_Nonprompt_All_MaxWeightTTVA_pt500", "AnalysisMuonsAuxDyn.ptcone40",
    "AnalysisMuonsAuxDyn.ptvarcone30_Nonprompt_All_MaxWeightTTVA_pt1000_CloseByCorr",
    "AnalysisMuonsAuxDyn.ptvarcone30_Nonprompt_All_MaxWeightTTVA_pt1000", "AnalysisMuonsAuxDyn.ptvarcone40",
    "AnalysisElectronsAuxDyn.f1", "AnalysisMuonsAuxDyn.ptcone20_Nonprompt_All_MaxWeightTTVA_pt500",
    "PrimaryVerticesAuxDyn.vertexType", "AnalysisMuonsAuxDyn.ptvarcone30", "AnalysisMuonsAuxDyn.ptcone30",
    "AnalysisMuonsAuxDyn.ptcone20_Nonprompt_All_MaxWeightTTVA_pt1000",
    "AnalysisElectronsAuxDyn.ptvarcone30_Nonprompt_All_MaxWeightTTVALooseCone_pt500", "AnalysisMuonsAuxDyn.CaloLRLikelihood"
]

In [4]:
# for funcadl-uproot
# funcadl_query = FuncADLQuery()
# for b in BRANCH_LIST:
#     funcadl_query = funcadl_query.Select(lambda e: {b: e[b]})

# for atlasr22
funcadl_query = FuncADLQueryPHYSLITE()
funcadl_query = funcadl_query.Select(lambda e: {
        "evt": e.EventInfo("EventInfo"),
        "jet": e.Jets(),
    })
funcadl_query = funcadl_query.Select(lambda ei: {
            "event_number": ei.evt.eventNumber(),  # type: ignore
            "run_number": ei.evt.runNumber(),  # type: ignore
            "jet_pt": ei.jet.Select(lambda j: j.pt() / 1000),  # type: ignore
            "jet_eta": ei.jet.Select(lambda j: j.eta()),  # type: ignore
            "jet_phi": ei.jet.Select(lambda j: j.phi()),  # type: ignore
            "jet_m": ei.jet.Select(lambda j: j.m()),  # type: ignore
            "jet_EnergyPerSampling":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vfloat]("EnergyPerSampling")
                ),
            "jet_SumPtTrkPt500":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vfloat]("SumPtTrkPt500")
                ),
            "jet_TrackWidthPt1000":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vfloat]("TrackWidthPt1000")
                ),
            "jet_NumTrkPt500":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vint]("NumTrkPt500")
                ),
            "jet_NumTrkPt1000":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vint]("NumTrkPt1000")
                ),
            "jet_SumPtChargedPFOPt500":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_vfloat]("SumPtChargedPFOPt500")
                ),
            "jet_Timing":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("Timing")
                ),
            "jet_JetConstitScaleMomentum_eta":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("JetConstitScaleMomentum_eta")
                ),
            "jet_ActiveArea4vec_eta":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("ActiveArea4vec_eta")
                ),
            "jet_DetectorEta":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("DetectorEta")
                ),
            "jet_JetConstitScaleMomentum_phi":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("JetConstitScaleMomentum_phi")
                ),
            "jet_ActiveArea4vec_phi":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("ActiveArea4vec_phi")
                ),
            "jet_JetConstitScaleMomentum_m":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("JetConstitScaleMomentum_m")
                ),
            "jet_JetConstitScaleMomentum_pt":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("JetConstitScaleMomentum_pt")
                ),
            "jet_Width":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("Width")
                ),
            "jet_EMFrac":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("EMFrac")
                ),
            "jet_ActiveArea4vec_m":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("ActiveArea4vec_m")
                ),
            "jet_DFCommonJets_QGTagger_TracksWidth":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("DFCommonJets_QGTagger_TracksWidth")
                ),
            "jet_JVFCorr":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("JVFCorr")
                ),
            "jet_DFCommonJets_QGTagger_TracksC1":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("DFCommonJets_QGTagger_TracksC1")
                ),
            "jet_PSFrac":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("PSFrac")
                ),
            "jet_DFCommonJets_QGTagger_NTracks":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_int]("DFCommonJets_QGTagger_NTracks")
                ),
            "jet_DFCommonJets_fJvt":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("DFCommonJets_fJvt")
                ),
            "jet_PartonTruthLabelID":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_int]("PartonTruthLabelID")
                ),
            "jet_HadronConeExclExtendedTruthLabelID":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_int]("HadronConeExclExtendedTruthLabelID")
                ),
            "jet_ConeTruthLabelID":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_int]("ConeTruthLabelID")
                ),
            "jet_HadronConeExclTruthLabelID":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_int]("HadronConeExclTruthLabelID")
                ),
            "jet_ActiveArea4vec_pt":
                ei.jet.Select(  # type: ignore
                    lambda j: j.getAttribute[cpp_float]("ActiveArea4vec_pt")
                ),
        })


In [7]:
# -------------------
# INPUT CONFIGURATION
# -------------------
# modify this to change how many files are being processed
# top-level processes determine containers/DSIDs, which each have some number of files
# full list is list(find_containers.container_dict.keys()) + ["data15_13TeV", "data16_13TeV", "data17_13TeV", "data18_13TeV"]

PROCESSES_TO_USE = ["ttbar"]  # 6.7 TB
# PROCESSES_TO_USE = ["db", "zjets", "wjets", "ttV", "othertop", "ttbar"]  # all simulation, 48.4 TB
# PROCESSES_TO_USE = ["db", "zjets", "wjets", "ttV", "othertop", "ttbar", "data15_13TeV", "data16_13TeV", "data17_13TeV", "data18_13TeV"]  # 191 TB

fileset = utils.get_fileset(PROCESSES_TO_USE, max_files_per_container=10, max_containers_per_dsid=None, max_dsid_per_process=None)
ofileset = fileset
# print(fileset)

newfset = { _ : {} for _ in fileset }
REMOTE_ACCESS = True
PARQUET = False
QUERYLANG = 'funcadl-xaod'

if QUERYLANG == 'uproot':
    Codegen = 'uproot-raw'
    Query = UprootRawQuery({'treename': 'CollectionTree', 'filter_name': BRANCH_LIST})
    ServiceX = 'testing4'
    Tree = 'CollectionTree'
elif QUERYLANG == 'funcadl-xaod':
    Codegen = 'atlasr22'
    Query = funcadl_query
    ServiceX = 'main'
    Tree = 'nominal'
elif QUERYLANG == 'funcadl-uproot':
    Codegen = 'uproot'
    Query = funcadl_query
    ServiceX = 'main'
    Tree = 'CollectionTree'

spec = ServiceXSpec(
    General=General(
        ServiceX=ServiceX,
        Codegen=Codegen,
        OutputFormat=('parquet' if PARQUET else 'root-file'),
        Delivery=('SignedURLs' if REMOTE_ACCESS else 'LocalCache'),
    ),
    Sample=[Sample(Name=process,
                     XRootDFiles=[strip_xcache(_) for _ in process_vals['files'].keys()],
                     Tree=Tree,
                     Query=Query,
                     IgnoreLocalCache=True,
                   # Codegen='atlasr22',
                    )
           for process, process_vals in fileset.items()],
)
data = deliver(spec)
for process, d in data.items():
    if PARQUET:
        newfset[process]['files'] = [_ for _ in d]
    else:
        if QUERYLANG == 'funcadl-xaod':
            newfset[process]['files'] = {_: 'atlas_xaod_tree' for _ in d}
        else:
            newfset[process]['files'] = {_: 'CollectionTree' for _ in d}
fileset = newfset

fileset summary
 - number of files: 30
cannot determine total size / number of events when max_files_per_container is being used


Returning code generators from cache

Output()

ServiceX Transform e140785e-e598-4b49-96dd-882bbe29e1c3

Transforms completed successfully

In [8]:
# check for files not yet replicated to MWT2 - kind of beside the point
files_at_mwt2 = 0
files_elsewhere = 0
for process in ofileset.keys():
    for file in ofileset[process]["files"]:
        if "mwt2" in file:
            files_at_mwt2 += 1
        else:
            files_elsewhere += 1

print(f"files at MWT2: {files_at_mwt2}, elsewhere: {files_elsewhere}")

files at MWT2: 30, elsewhere: 0


In [9]:
def raw_materialize_branches(events, remap=True):
    num_events = ak.num(events, axis=0)  # track number of events

    # this will read around 25% of data files
    # materialize branches, just derive integers from them that will be aggregated to avoid memory issues
    _counter = 0
    for branch in BRANCH_LIST:
        if remap:
            obj_name, obj_prop = branch.split(".")
            obj_name = obj_name.replace("Analysis", "").replace("AuxDyn", "")
            if "Link" not in obj_prop:
                branch_data = events[obj_name, obj_prop]
            else:
                branch_data = events[obj_name, obj_prop]["m_persIndex"]
        else:
            obj_name = branch.replace('*', '')
            if "Link" not in obj_name:
                branch_data = events[obj_name]
            # else:
            #     branch_data = events[f'{obj_name}.m_persIndex']           
                _counter += ak.count_nonzero(branch_data)

    return {"nevts": num_events, "_counter": _counter}

def funcadl_materialize_branches(data, remap=True):
    num_events = ak.num(data, axis=0)  # track number of events
    total_count = 0
    for field in data.fields:
        logging.debug(f"Counting field {field}")
        if str(data[field].type.content).startswith("var"):
            count = ak.count_nonzero(data[field], axis=-1)
            for _ in range(count.ndim - 1):  # type: ignore
                count = ak.count_nonzero(count)

            total_count = total_count + count  # type: ignore
        else:
            # We get a not implemented error when we try to do this
            # on leaves like run-number or event-number (e.g. scalars)
            # Maybe we should just be adding a 1. :-)
            logging.info(f"Field {field} is not a scalar field. Skipping count.")

    # total_count = ak.count_nonzero(total_count, axis=0)
    return {"nevts": num_events, "_counter": total_count}

if QUERYLANG == 'funcadl-xaod':
    materialize_branches = funcadl_materialize_branches
else:
    materialize_branches = raw_materialize_branches

# following cells are for ROOT output

In [10]:
%%time
# from coffea.dataset_tools.preprocess import _normalize_file_info
# # print(fileset)
# print(ak.from_iter(_normalize_file_info(fileset['ttbar']['files'])))
# pre-process
# samples, report = dataset_tools.preprocess(fileset, skip_bad_files=True, uproot_options={"allow_read_errors_with_report": True})
samples, report = dataset_tools.preprocess(fileset, skip_bad_files=False)

CPU times: user 16.9 s, sys: 2.08 s, total: 19 s
Wall time: 2min 12s


In [11]:
# find issues where access did not work
for process in report:
    for k, v in report[process]["files"].items():
        if v["steps"] is None:
            print(f"could not read {k}")

In [12]:
%%time
# create the task graph
# filter_name seems to not do anything here in terms of performance
filter_name = lambda name: name in BRANCH_LIST
tasks = dataset_tools.apply_to_fileset(materialize_branches,
                                       samples,
                                       uproot_options={"allow_read_errors_with_report": (OSError, TypeError), "filter_name": filter_name},
                                       schemaclass=PHYSLITESchema)

CPU times: user 2 s, sys: 317 ms, total: 2.32 s
Wall time: 3.76 s


In [13]:
%%time
# execute task graph
t0 = time.perf_counter()
with performance_report(filename="dask-report.html"):
    ((out, report),) = dask.compute(tasks)  # feels strange that this is a tuple-of-tuple
t1 = time.perf_counter()

print(f"total time spent in uproot reading data: {ak.sum([v['duration'] for v in report.values()]):.2f} s")
print(f"wall time: {t1-t0:.2f}s")

total time spent in uproot reading data: 33.19 s
wall time: 8.90s
CPU times: user 5.71 s, sys: 768 ms, total: 6.48 s
Wall time: 8.91 s


In [14]:
print(f"output: {out}")

print("\nperformance metrics:")
event_rate = sum([out[process]["nevts"] for process in out.keys()]) / (t1-t0)
print(f" - event rate: {event_rate / 1_000:.2f} kHz")

# need uproot>=5.3.2 to get these useful performance stats
num_bytes = ak.sum([report[process]["performance_counters"]["num_requested_bytes"] for process in out.keys()])
read_MB = num_bytes / 1_000**2
rate_Mbs = read_MB / (t1-t0)
print(f" - read {read_MB:.2f} MB in {t1-t0:.2f} s -> {rate_Mbs*8:.2f} Mbps (need to scale by x{200/8/rate_Mbs*1000:.0f} to reach 200 Gbps)")

output: {'ttbar': {'nevts': 650000, '_counter': 0}}

performance metrics:
 - event rate: 72.99 kHz
 - read 0.84 MB in 8.90 s -> 0.75 Mbps (need to scale by x266434 to reach 200 Gbps)


In [15]:
# report problematic files that caused exceptions
for process in report.keys():
    for i_file in range(len(report[process].exception)):
        file_report = report[process][i_file]
        if file_report.exception is not None:
            print(file_report.args[0].strip("\'"))
            print(file_report.message + "\n")

In [12]:
# sanity check that the right colums are being touched
# dak.report_necessary_columns(tasks)

In [13]:
# if issues with files exist, paste in path and reproduce
# fname = "root://192.170.240.146:1094//root://fax.mwt2.org:1094//pnfs/uchicago.edu/atlaslocalgroupdisk/rucio/mc20_13TeV/59/28/DAOD_PHYSLITE.37231868._000040.pool.root.1"
# treename = "CollectionTree"
# events = NanoEventsFactory.from_root({fname: treename}, schemaclass=PHYSLITESchema).events()
# task = materialize_branches(events)
# task["_counter"].compute()

# following cells are for parquet output

In [37]:
# This will fail for now on uproot output due to https://github.com/dask-contrib/dask-awkward/issues/501

# events = NanoEventsFactory.from_parquet(fileset['ttbar']['files'][0], schemaclass=PHYSLITESchema)
# task = materialize_branches(events)
# task["_counter"].compute()
print(fileset['ttbar']['files'])
ds = dak.from_parquet(fileset['ttbar']['files'], behavior=PHYSLITESchema.behavior())
# filter_name = lambda name: name in BRANCH_LIST
# ds.map_partitions(materialize_branches)
task = materialize_branches(ds, False)
task["_counter"].compute()

[]


ValueError: empty urlpath sequence

In [ ]:
fname = "root://fax.mwt2.org:1094//pnfs/uchicago.edu/atlaslocalgroupdisk/rucio/mc20_13TeV/59/28/DAOD_PHYSLITE.37231868._000040.pool.root.1"
treename = "CollectionTree"
events = NanoEventsFactory.from_root({fname: treename}, schemaclass=PHYSLITESchema).events()
task = materialize_branches(events)
task["_counter"].compute()

In [42]:
print(data)

{'ttbar': []}
